## Use SX5E and bloomberg download data to get me started

In [1]:
import pandas as pd
import numpy as np


In [ ]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0])
    return adj

In [43]:
#pull the data from excel
tabs_to_grab=['Quarter','Date','Reported EPS','Comp EPS','Consensus EPS','50d MA','3d return','Volume','Momentum',
             'Revision','Market cap','PE','property','Price']

data_dict={}

for s in tabs_to_grab:
    data_dict[s]=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result code\Catalyst.xlsx',sheetname=s))


C:\Users\Eric.Li\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\util\_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [51]:
#signal formation
signal=data_dict['Date'][(data_dict['3d return']>2) & (data_dict['Revision']>0.02)]
signal=signal.dropna(axis=1,how='all')
signal=signal.dropna(axis=0,how='all')
signal

,2018-03-07 00:00:00,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
Unnamed: 0,,,,,,,,,
AD NA,2018-02-28,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
ADS GY,NaT,NaT,NaT,2017-03-08,NaT,NaT,NaT,NaT,NaT
AIR FP,2018-02-15,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
ALV GY,NaT,NaT,NaT,NaT,2017-02-16,NaT,NaT,NaT,NaT
ASML NA,2018-01-17,NaT,2017-07-19,NaT,NaT,2016-10-19,NaT,NaT,NaT
BAS GY,NaT,2017-10-24,NaT,NaT,NaT,NaT,NaT,NaT,NaT
BMW GY,NaT,2017-08-03,2017-05-04,NaT,NaT,NaT,NaT,NaT,NaT
CRH ID,NaT,NaT,NaT,2016-08-25,NaT,NaT,NaT,NaT,NaT
DG FP,NaT,NaT,2017-02-07,NaT,NaT,NaT,NaT,NaT,NaT


In [ ]:
#Performance analytics